<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/NetworkDesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.1.3 which is incompatible.
tensorflow-metadata 1.14.0 

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Daten aus csv-Datei laden



In [ ]:
! git clone https://github.com/AlexKressner/Industrielles_Management

Cloning into 'Industrielles_Management'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 53 (delta 16), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (53/53), 24.73 KiB | 3.53 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
# Daten laden
import pandas as pd

In [ ]:
folder = "Industrielles_Management/Daten/NetworkDesign"

In [ ]:
path = f"{folder}/NetworkDesign_Invest_Kapa.csv"

In [ ]:
kapa=pd.read_csv(path, sep=";")

In [ ]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [ ]:
path = f"{folder}/NetworkDesign_Produktionskosten.csv"

In [ ]:
produktionskosten=pd.read_csv(path, sep=";")

In [ ]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [ ]:
path = f"{folder}/NetworkDesign_Nachfrage.csv"

In [ ]:
nachfrage=pd.read_csv(path, sep=";")

In [ ]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Indexmengen

In [ ]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [ ]:
I

['Charleston', 'Curitiba', 'Hamburg', 'Quanzhou', 'East London']

In [ ]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [ ]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika']

In [ ]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [ ]:
A

['klein', 'groß']

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Definition der Entscheidungsvariablen
# x sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
x={}
for i in I:
  for j in J:
    x[i,j] = solver.IntVar(0, infinity, f"{i},{j}")

In [ ]:
# Definition der Entscheidungsvariablen
# y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
y={}
for i in I:
  for a in A:
    y[i,a] = solver.BoolVar(f"{i},{a}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 35


## Parameter

In [ ]:
# Herstell- und Transportkosten
produktionskosten.set_index(["Produktionsstandort","Markt"], inplace=True)

In [ ]:
c = produktionskosten.to_dict("dict")["Produktionskosten"]

In [ ]:
c

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

In [ ]:
# Nachfrage aus den Märtken

In [ ]:
nachfrage.set_index(["Markt"], inplace=True)

In [ ]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [ ]:
d

{'USA': 12, 'Südamerika': 8, 'Europa': 14, 'Asien': 16, 'Afrika': 7}

In [ ]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapazitaeten = kapa.set_index("Ausbaustufe")

In [ ]:
cap = kapazitaeten.to_dict("dict")["Kapazitäten"]

In [ ]:
cap

{'klein': 10, 'groß': 20}

In [ ]:
f = kapa.set_index(["Produktionsstandort","Ausbaustufe"]).to_dict("dict")["Investitionen"]

In [ ]:
f

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

## Zielfunktion

min $K=\sum_{i,j} c_{ij}* X_{ij} + \sum_{i,a} fc_{ia}* Y_{ia}$

In [ ]:
# Maximierung der gesamten Deckungsbeiträge
solver.Minimize(sum(c[i,j]*x[i,j] for i in I for j in J) + sum(f[i,a]*y[i,a] for i in I for a in A))

## Nebenbedingungen

$\sum_i X_{ij} \ge d_j, ∀j$  

In [ ]:
# Nachfrage in den Märkten erfüllen
for j in J:
  solver.Add(sum(x[i,j] for i in I)>=d[j])

$\sum_j X_{ij} \le \sum_a cap_{ia} * Y_{ia}, ∀i$

In [ ]:
# Kapazitäten aufbauen und einhalten
for i in I:
  solver.Add(sum(x[i,j] for j in J)<= sum(cap[a]*y[i,a] for a in A))

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
    print("\n")
    print("Produktions-Markt-Zuteilung")
    for j in J:
      print(f"Markt:{j}")
      for i in I:
        if x[i,j].solution_value()>0:
          print(f'{i,j} =', round(x[i,j].solution_value()))
    print("\n")
    print("Invest Standorte")
    for i in I:
      for a in A:
        if y[i,a].solution_value()>0:
          print(f'{i,a} =', round(y[i,a].solution_value()))
    print("\n")
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 33156.99999999999


Produktions-Markt-Zuteilung
Markt:USA
('Charleston', 'USA') = 10
('Hamburg', 'USA') = 2
Markt:Südamerika
('Hamburg', 'Südamerika') = 8
Markt:Europa
('Hamburg', 'Europa') = 14
Markt:Asien
('Quanzhou', 'Asien') = 16
Markt:Afrika
('Hamburg', 'Afrika') = 3
('Quanzhou', 'Afrika') = 4


Invest Standorte
('Charleston', 'klein') = 1
('Hamburg', 'klein') = 1
('Hamburg', 'groß') = 1
('Quanzhou', 'groß') = 1


